In [16]:
import pandas as pd
import sqlite3
import os
import glob

csv_folder = "data/brazilian-ecommerce/versions/2"
db_path = 'data/commerce.db'

conn = sqlite3.connect(db_path)
print(f"Verbindung zur {db_path} Datenbank hergestellt.")

all_files = glob.glob(os.path.join(csv_folder, '*.csv'))

print(f"Gefundene CSV-Dateien: {all_files}")
print("-"*50)

for filename in all_files:
    base_name = os.path.basename(filename)
    table_name = base_name.replace('.csv', '')

    try:
        print(f"Verarbeite: {table_name}...")
        df = pd.read_csv(filename)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        count = pd.read_sql_query(f"SELECT count(*) FROM {table_name}", conn).iloc[0,0]
        print(f"   ✅ Erfolgreich! {count} Zeilen importiert.")
    except Exception as e:
        print(f"   ❌ Fehler beim Importieren von {table_name}: {e}")

print("-"*50)
print("Alle Importe abgeschlossen.")

Verbindung zur data/commerce.db Datenbank hergestellt.
Gefundene CSV-Dateien: ['data/brazilian-ecommerce/versions/2\\olist_customers_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_geolocation_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_orders_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_order_items_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_order_payments_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_order_reviews_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_products_dataset.csv', 'data/brazilian-ecommerce/versions/2\\olist_sellers_dataset.csv', 'data/brazilian-ecommerce/versions/2\\product_category_name_translation.csv']
--------------------------------------------------
Verarbeite: olist_customers_dataset...
   ✅ Erfolgreich! 99441 Zeilen importiert.
Verarbeite: olist_geolocation_dataset...
   ✅ Erfolgreich! 1000163 Zeilen importiert.
Verarbeite: olist_orders_dataset...
   ✅ Erfolgreich! 99441 Zeilen import

### Level 1: Analyzing Low Review Scores
Das Customer-Success-Team möchte wissen, warum Kunden unzufrieden sind. Sie wollen eine Liste aller "schlechten" Bewertungen (Score 1 oder 2). Das Problem: Viele Kunden geben nur Sterne, schreiben aber keinen Text. Das Team will diese unterscheiden.

In [18]:
query_level_1 = """
select
    review_id, review_score,
    case
        when review_comment_message is null then 'no_comment'
        else 'has_comment'
    end as review_category
from
    olist_order_reviews_dataset
where
    review_score <= 2
order by review_score asc
"""

df_level_1 = pd.read_sql_query(sql=query_level_1, con=conn)
df_level_1.head(10)

,review_id,review_score,review_category
0,15197aa66ff4d0650b5434f1b46cda19,1,no_comment
1,373cbeecea8286a2b66c97b1b157ec46,1,has_comment
2,2c5e27fc178bde7ac173c9c62c31b070,1,has_comment
3,58044bca115705a48fe0e00a21390c54,1,has_comment
4,9fd59cd04b42f600df9f25e54082a8d1,1,has_comment
5,eb26c2bfb5030f57dcef30d3f111eb1e,1,no_comment
6,e233e51d11511bf30e568c76360ace52,1,has_comment
7,6d06808638ec0701bccd70bc8d462c28,1,has_comment
8,60c714ed14cef913944a3147094a4742,1,has_comment
9,65dfeb60c40e3cbb0a1838285d86f885,1,has_comment


### Level 2: Big Player?
Das Management will wissen, welche Verkäufer (Sellers) den meisten Umsatz machen. Aber wir interessieren uns nur für die "Wale" – also Verkäufer, die richtig viel umsetzen.

In [19]:
query_level_2 = """
select
    s.seller_id,
    s.seller_state,
    count(distinct i.order_id) as total_orders,
    sum(i.price + i.freight_value) as total_revenue
from olist_order_items_dataset i
join
    olist_sellers_dataset s
    on i.seller_id = s.seller_id
group by
    s.seller_id,
    s.seller_state
having
    total_revenue > 100000.0
order by
    total_revenue desc
"""

df_level_2 = pd.read_sql_query(sql=query_level_2, con=conn)
df_level_2.head(10)

,seller_id,seller_state,total_orders,total_revenue
0,4869f7a5dfa277a7dca6462dcf3b52b2,SP,1132,249640.70
1,7c67e1448b00f6e969d365cea6b010ab,SP,982,239536.44
2,53243585a1d6dc2643021fd1853d8905,BA,358,235856.68
3,4a3ca9315b744ce9f8e9374361493884,SP,1806,235539.96
4,fa1c13f2614d7b5c4749cbc52fecda94,SP,585,204084.73
5,da8622b14eb17ae2831f4ac5b9dab84a,SP,1314,185192.32
6,7e93a43ef30c4f03f38b393420bc753a,SP,336,182754.05
7,1025f0e2d44d7041d6cf58b6550e0bfa,SP,915,172860.69
8,7a67c85e85bb2ce8582c35f2203ad736,SP,1160,162648.38
9,955fee9216a65b617aa5c0531780ce60,SP,1287,160602.68


### Level 3: Die Zeitreise
Im Interview kommt fast immer eine Frage zu Zeitreihen. "Wie entwickelt sich die Lieferzeit über die Monate?" oder "Wie hoch ist die Retention Rate pro Quartal?"
Das Problem: Computer verstehen Datumswerte oft nur als Text. Du musst lernen, wie man:
- Aus einem Datum den Monat extrahiert (z.B. "2017-01" aus "2017-01-15").
- Die Differenz zwischen zwei Daten berechnet (Lieferdauer = Lieferdatum - Kaufdatum).

Deine Aufgabe: Analysiere die Logistik-Performance.

In [21]:
query_level_3 = """
select strftime('%Y-%m', o.order_purchase_timestamp) as order_month,
        avg(julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp)) as avg_delivery_days, count(*) as total_orders
from olist_orders_dataset o
where
    order_delivered_customer_date is not null and
    strftime('%Y', order_purchase_timestamp) >= '2017'
group by order_month
order by order_month asc
"""
df_level_3 = pd.read_sql_query(sql=query_level_3, con=conn)
df_level_3.head(10)

,order_month,avg_delivery_days,total_orders
0,2017-01,12.647044,750
1,2017-02,13.168825,1653
2,2017-03,12.951184,2546
3,2017-04,14.917913,2303
4,2017-05,11.322363,3545
5,2017-06,12.011573,3135
6,2017-07,11.592732,3872
7,2017-08,11.147125,4193
8,2017-09,11.851007,4150
9,2017-10,11.856700,4478


### Level 4: The Boss Fight - Window Functions
Was sind die Top 3 Produkte (nach Umsatz) in JEDER Kategorie?

In [23]:
query_level_4  = """
with product_sales as (
    select
        p.product_category_name,
        i.product_id,
        sum(i.price + i.freight_value) as total_revenue
    from olist_order_items_dataset i
    join olist_products_dataset p
        on i.product_id = p.product_id
    where p.product_category_name is not null
    group by
        p.product_category_name,
        i.product_id
)
select
    product_category_name,
    product_id,
    total_revenue,
    rank() over (partition by product_category_name order by total_revenue desc) as revenue_rank
from product_sales
order by product_category_name, revenue_rank
"""
df_level_4 = pd.read_sql_query(sql=query_level_4, con=conn)
df_level_4.head(10)

,product_category_name,product_id,total_revenue,revenue_rank
0,agro_industria_e_comercio,11250b0d4b709fee92441c5f34122aed,9710.48,1
1,agro_industria_e_comercio,423a6644f0aa529e8828ff1f91003690,8609.73,2
2,agro_industria_e_comercio,672e757f331900b9deea127a2a7b79fd,7355.34,3
3,agro_industria_e_comercio,c183fd5d2abf05873fa6e1014ed9e06c,6081.54,4
4,agro_industria_e_comercio,2b69866f22de8dad69c976771daba91c,3184.55,5
5,agro_industria_e_comercio,c89226b8a795ae3d6bca9d90b20dbf04,2893.49,6
6,agro_industria_e_comercio,5fb0955cb683eb6f65a1f613e502eef5,2776.11,7
7,agro_industria_e_comercio,b7a60a397d4efd05c1b5d398fb9f9097,2467.33,8
8,agro_industria_e_comercio,cd5df6a3db7a3d064a55afd08289d762,2398.36,9
9,agro_industria_e_comercio,cd2f5c10e4e8dbc701f0bb68a09fdfe8,2315.83,10


### Level 5: RFM Analysis & Customer Segmentation
Was ist RFM? Im Marketing teilt man Kunden in Gruppen ein, um Budget effizient zu nutzen.

- Recency: Wie viele Tage sind seit dem letzten Kauf vergangen? (Je kleiner, desto besser)
- Frequency: Wie oft hat der Kunde gekauft? (Je höher, desto besser)
- Monetary: Wie viel Umsatz hat er generiert? (Je höher, desto besser)

Deine Aufgabe: Simuliere, dass heute der 30. August 2018 ist

In [28]:
query_level_5 = """
with customer_metrics as (
    select
        c.customer_unique_id,
        cast(julianday('2018-08-30') - julianday(max(o.order_purchase_timestamp)) as integer) as recency,
        count(distinct o.order_id) as frequency,
        sum(i.price + i.freight_value) as monetary
    from olist_orders_dataset o
    join olist_order_items_dataset i
        on o.order_id = i.order_id
    join olist_customers_dataset c
        on o.customer_id = c.customer_id
    where o.order_purchase_timestamp <= '2018-08-30'
    group by c.customer_unique_id
)
select
    customer_unique_id,
    recency,
    frequency,
    monetary,
    case
        when recency < 90 and monetary > 500 then 'VIP'
        when recency > 300  then 'Churned'
        else 'Regular'
    end as customer_segment
from customer_metrics
order by monetary desc
"""

df_level_5 = pd.read_sql_query(query_level_5, con=conn)
df_level_5.head(10)

,customer_unique_id,recency,frequency,monetary,customer_segment
0,0a0a92112bd4c708ca5fde585afaa872,334,1,13664.08,Churned
1,da122df9eeddfedc1dc1f5349a1a690c,515,2,7571.63,Churned
2,763c8b1c9c68a0229c42c9fc6f662b93,45,1,7274.88,VIP
3,dc4802a71eae9be1dd28f5d788ceb526,563,1,6929.31,Churned
4,459bef486812aa25204be022145caa62,35,1,6922.21,VIP
5,ff4159b92c40ebe40454e3e6a7c35ed6,462,1,6726.66,Churned
6,4007669dec559734d6f53e029e360987,278,1,6081.54,Regular
7,5d0a2980b292d049061542014e8960bf,48,1,4809.44,VIP
8,eebb5dda148d3893cdaf5b5ca3040ccb,498,1,4764.34,Churned
9,48e1ac109decbb87765a3eade6854098,68,1,4681.78,VIP
